In [122]:
import pandas as pd
import numpy as np

In [123]:
df = pd.read_csv("/content/new_df.csv")

In [124]:
df.head()

,movieId,title,genres,imdbId,tmdbId,rating,tag,new_genres,new_title,text,transformed_text
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,3.920930,"pixar,pixar,fun",Adventure Animation Children Comedy Fantasy,Toy Story 1995,Toy Story 1995 Adventure Animation Children C...,toy stori 1995 adventur anim children comedi f...
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,3.431818,"fantasy,magic board game,Robin Williams,game",Adventure Children Fantasy,Jumanji 1995,Jumanji 1995 Adventure Children Fantasy,jumanji 1995 adventur children fantasi
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,3.259615,"moldy,old",Comedy Romance,Grumpier Old Men 1995,Grumpier Old Men 1995 Comedy Romance,grumpier old men 1995 comedi romanc
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,2.357143,NaN,Comedy Drama Romance,Waiting to Exhale 1995,Waiting to Exhale 1995 Comedy Drama Romance,wait exhal 1995 comedi drama romanc
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,3.071429,"pregnancy,remake",Comedy,Father of the Bride Part II 1995,Father of the Bride Part II 1995 Comedy,father bride part ii 1995 comedi


In [125]:
## implementing KNN from scratch
class KNN:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def recommend(self, X_test):
        predictions = []
        for sample in X_test:
            # print("checking")
            distances = []
            for i in range(len(self.X_train)):
                distance = np.sqrt(np.sum(np.square(sample - self.X_train[i])))  ## calculating the distances
                distances.append((distance, self.y_train[i]))
            distances.sort(key=lambda x: x[0])  ## sorting the distances
            neighbors = distances[:self.k + 1]  ## finding the neighbours
            classes = [neighbor[1] for neighbor in neighbors]
            # prediction = max(set(classes), key=classes.count)
            predictions.append(classes)
        return predictions  ## returning the predictions

In [126]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(max_features=100,stop_words='english')

In [127]:
vector = cv.fit_transform(df['transformed_text']).toarray()

In [128]:
vector.shape

(9742, 100)

In [129]:
knn = KNN(k = 5)

In [130]:
title = df["title"].values

In [131]:
knn.fit(vector, title)

In [132]:
def recommend(movieTitle):
  movieId = df[df["title"] == movieTitle]["movieId"].iloc[0] - 1
  movieVector = vector[movieId, :]
  movieVec = movieVector.reshape(1, -1)
  recommendations = knn.recommend(movieVec)
  return recommendations[0][1 :]


In [133]:
recommend("Jumanji (1995)")

['Indian in the Cupboard, The (1995)',
 'Tom and Huck (1995)',
 'Casper (1995)',
 'Escape to Witch Mountain (1975)',
 'Tall Tale (1995)']